# OllamaFunctions | OllamaFunctions

このノートブックは、OllamaにOpenAI Functionsと同じAPIを提供する実験的なラッパーの使用方法を示しています。

> This notebook shows how to use an experimental wrapper around Ollama that gives it the same API as OpenAI Functions.

より強力で高機能なモデルは、複雑なスキーマや複数の機能を持つ場合により良いパフォーマンスを発揮します。以下の例ではMistralを使用しています。サポートされているモデルとモデルのバリアントの完全なリストについては、[Ollamaモデルライブラリ](https://ollama.ai/library)をご覧ください。

> Note that more powerful and capable models will perform better with complex schema and/or multiple functions. The examples below use Mistral.
> For a complete list of supported models and model variants, see the [Ollama model library](https://ollama.ai/library).

## Setup | セットアップ

ローカルのOllamaインスタンスをセットアップして実行するには、[この手順](https://github.com/jmorganca/ollama)に従ってください。

> Follow [these instructions](https://github.com/jmorganca/ollama) to set up and run a local Ollama instance.

## Usage | 使用方法

OllamaFunctionsは、標準のChatOllamaインスタンスを初期化するのと同じような方法で初期化できます：

> You can initialize OllamaFunctions in a similar way to how you'd initialize a standard ChatOllama instance:



In [1]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(model="mistral")

JSONスキーマパラメータと`function_call`パラメータを使用して定義された関数をバインドすることで、モデルに指定された関数を強制的に呼び出させることができます。

> You can then bind functions defined with JSON Schema parameters and a `function_call` parameter to force the model to call the given function:



In [2]:
model = model.bind(
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

このモデルで関数を呼び出すと、提供されたスキーマに一致するJSON出力が得られます。

> Calling a function with this model then results in JSON output matching the provided schema:



In [3]:
from langchain.schema import HumanMessage

model.invoke("what is the weather in Boston?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Boston, MA", "unit": "celsius"}'}})

## Using for extraction | 抽出用に使用する

ここで関数呼び出しを使って行う便利なことの一つは、与えられた入力から構造化された形式でプロパティを抽出することです：

> One useful thing you can do with function calling here is extracting properties from a given input in a structured format:



In [4]:
from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "name": {"type": "string"},
        "height": {"type": "integer"},
        "hair_color": {"type": "string"},
    },
    "required": ["name", "height"],
}

# Input
input = """Alex is 5 feet tall. Claudia is 1 feet taller than Alex and jumps higher than him. Claudia is a brunette and Alex is blonde."""

# Run chain
llm = OllamaFunctions(model="mistral", temperature=0)
chain = create_extraction_chain(schema, llm)
chain.run(input)

[{'name': 'Alex', 'height': 5, 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': 6, 'hair_color': 'brunette'}]